In [1]:
SEED = 61


import os
import re
import gc
import h5py
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA

from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPooling2D
from keras.layers.core import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [2]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')
    
    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [3]:
def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]
        
    return labels, text_labels, image_labels

In [4]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 3, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 3, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 3, 3)
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('F1-SCORE')
    plt.ylabel('f1-score')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [5]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * data.shape[0])
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

In [6]:
def create_model_pretrained(input_shape):
    f1_score = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='micro')
    
    image_input = Input(shape=input_shape)
    outputs = Dense(NUM_CLASSES, activation='softmax') (image_input)
    
    model = Model(image_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score]) # f1 #tf.keras.metrics.AUC()
    return model

In [7]:
def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))#, custom_objects={'f1': f1})
    
    loss, acc, f1 = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('F1-score:', f1)
        
    return loss, acc, f1

In [8]:
def run_and_evaluate(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    model = create_model_pretrained(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                                   epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                                   callbacks=[checkpoint, early_stopping])
    
    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

# Load data

In [9]:
# load separate
mvsa_single_vgg16, mvsa_multiple_vgg16 = load_mvsa_feature('vgg16')
mvsa_single_vgg19, mvsa_multiple_vgg19 = load_mvsa_feature('vgg19')
mvsa_single_resnet50, mvsa_multiple_resnet50 = load_mvsa_feature('resnet50')
mvsa_single_resnet101, mvsa_multiple_resnet101 = load_mvsa_feature('resnet101')
mvsa_single_resnet152, mvsa_multiple_resnet152 = load_mvsa_feature('resnet152')
mvsa_single_densenet121, mvsa_multiple_densenet121 = load_mvsa_feature('densenet121')
mvsa_single_densenet169, mvsa_multiple_densenet169 = load_mvsa_feature('densenet169')
mvsa_single_densenet201, mvsa_multiple_densenet201 = load_mvsa_feature('densenet201')

In [10]:
# load merge
mvsa_vgg16 = load_mvsa_feature('vgg16', merge=True)
mvsa_vgg19 = load_mvsa_feature('vgg19', merge=True)
mvsa_resnet50 = load_mvsa_feature('resnet50', merge=True)
mvsa_resnet101 = load_mvsa_feature('resnet101', merge=True)
mvsa_resnet152 = load_mvsa_feature('resnet152', merge=True)
mvsa_densenet121 = load_mvsa_feature('densenet121', merge=True)
mvsa_densenet169 = load_mvsa_feature('densenet169', merge=True)
mvsa_densenet201 = load_mvsa_feature('densenet201', merge=True)

In [11]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, mvsa_single_image_labels = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, mvsa_multiple_image_labels = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

In [12]:
le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3

In [13]:
# prepare all features data
pretrained_names = ['vgg16', 'vgg19', 'resnet50', 'resnet101', 'resnet152', 'densenet121', 'densenet169', 'densenet201']
# single_pretrained_names = ['single-' + name for name in pretrained_names]
# multiple_pretrained_names = ['multiple-' + name for name in pretrained_names]

mvsa_single_features = [mvsa_single_vgg16, mvsa_single_vgg19, 
                        mvsa_single_resnet50, mvsa_single_resnet101, mvsa_single_resnet152, 
                        mvsa_single_densenet121, mvsa_single_densenet169, mvsa_single_densenet201]

mvsa_multiple_features = [mvsa_multiple_vgg16, mvsa_multiple_vgg19, 
                          mvsa_multiple_resnet50, mvsa_multiple_resnet101, mvsa_multiple_resnet152, 
                          mvsa_multiple_densenet121, mvsa_multiple_densenet169, mvsa_multiple_densenet201]

# Run models and Evalution display

In [14]:
reset_seeds()
EPOCHS = 100
BATCH_SIZE = 1000
VALIDATION_SPLIT = 0.1
EARLY_STOPPING = 10

In [15]:
# run model with all features data of MVSA-Single
mvsa_single_hitories = []
mvsa_single_score = []
for i in range(len(pretrained_names)):
    print('MVSA-Single:', pretrained_names[i])
    history, scores = run_and_evaluate('single-' + pretrained_names[i], mvsa_single_features[i], mvsa_single_image_labels, verbose=0)
    mvsa_single_hitories.append(history)
    mvsa_single_score.append(scores)
    print()

MVSA-Single: vgg16


2022-06-30 11:58:09.280643: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-06-30 11:58:09.509501: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)



MVSA-Single: vgg19

MVSA-Single: resnet50

MVSA-Single: resnet101

MVSA-Single: resnet152

MVSA-Single: densenet121

MVSA-Single: densenet169

MVSA-Single: densenet201



In [16]:
print('MVSA-Single')
df_single_scores = pd.DataFrame(mvsa_single_score, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)
df_single_scores.style.highlight_max(props='color:orange', axis = 0)

MVSA-Single


,Loss,Accuracy,F1-score
vgg16,0.867092,0.589327,0.589327
vgg19,0.871784,0.600928,0.600928
resnet50,0.926033,0.573086,0.573086
resnet101,0.937503,0.554524,0.554524
resnet152,0.954709,0.559165,0.559165
densenet121,0.914535,0.600928,0.600928
densenet169,0.924687,0.568446,0.568446
densenet201,0.875166,0.614849,0.614849


In [17]:
# run model with all features data of MVSA-Multiple
mvsa_multiple_hitories = []
mvsa_multiple_score = []
for i in range(len(pretrained_names)):
    print('MVSA-Multiple:', pretrained_names[i])
    history, scores = run_and_evaluate('multiple-' + pretrained_names[i], mvsa_multiple_features[i], mvsa_multiple_image_labels, verbose=0)
    mvsa_multiple_hitories.append(history)
    mvsa_multiple_score.append(scores)
    print()

MVSA-Multiple: vgg16

MVSA-Multiple: vgg19

MVSA-Multiple: resnet50

MVSA-Multiple: resnet101

MVSA-Multiple: resnet152

MVSA-Multiple: densenet121

MVSA-Multiple: densenet169

MVSA-Multiple: densenet201



In [18]:
print('MVSA-Multiple')
df_multiple_scores = pd.DataFrame(mvsa_multiple_score, columns=['Loss', 'Accuracy', 'F1-score'], index=pretrained_names)
df_multiple_scores.style.highlight_max(props='color:orange', axis = 0)

MVSA-Multiple


,Loss,Accuracy,F1-score
vgg16,0.913603,0.562882,0.562882
vgg19,0.925325,0.558608,0.558608
resnet50,0.912656,0.543956,0.543956
resnet101,0.918681,0.541514,0.541514
resnet152,0.962034,0.544567,0.544567
densenet121,0.889988,0.566545,0.566545
densenet169,0.889151,0.565934,0.565934
densenet201,0.888390,0.567766,0.567766
